# Customer Intent Challenge!

http://34.85.195.130:8501/

![](https://miro.medium.com/max/1024/1*e8v1xC0NTgoduh_ei9F7Pw.png)

Your analytics firm was hired to help prove out that customer intents can be predicted via ML models.  The client believes that if ML models can classify incoming support messages properly that there is an opportunity to sreamline operations and improve customer satisfaction.

In [ ]:
# get the repo for the data
! git clone https://github.com/Btibert3/BA820-Fall-2021.git

# the data are in the challenges folder, class 13

# you are training a model on the training set
# you are applying the model to the test set
# start small, get feedback, and iterate!

# a good wrapup to consider all of the themes we covered in class!

In [8]:
# installs
! pip install -U spacy
! pip install spacy-universal-sentence-encoder

     |████████████████████████████████| 207.1 MB 46 kB/s 
     |████████████████████████████████| 108 kB 28.6 MB/s 
     |████████████████████████████████| 5.8 MB 7.9 MB/s 
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 1.3 MB 9.3 MB/s 
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 463 kB 15.7 MB/s 
     |████████████████████████████████| 12.2 MB 4.1 MB/s 
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 126 kB 4.7 MB/s 
     |████████████████████████████████| 1.6 MB 22.7 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
  Using cached tensorboard_data_server-0.6.1-py3-none-macosx_10_9_x86_64.whl (3.5 MB)
  Using cached tensorboard_plugin_wit-1.8.0-py3-no

In [16]:
# imports
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import spacy
from spacy import cli

import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

from umap import UMAP

# make sure we have the imports
import numpy as np
import pandas as pd
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
eval = pd.read_csv('/Users/alexandervonschwerdtner/Desktop/BA820 - Unsupervised Machine Learning & Text Analytics/BA820-Fall-2021/data-challenges/13-customer-intent/eval.csv')
sample = pd.read_csv('/Users/alexandervonschwerdtner/Desktop/BA820 - Unsupervised Machine Learning & Text Analytics/BA820-Fall-2021/data-challenges/13-customer-intent/sample.csv')
train = pd.read_csv('/Users/alexandervonschwerdtner/Desktop/BA820 - Unsupervised Machine Learning & Text Analytics/BA820-Fall-2021/data-challenges/13-customer-intent/training.csv')

In [4]:
train.sample(3)

,message,domain,id
9402,lock all the doors,home,504c964c-96b8-447a-833f-cc495b62654f
12466,what time is it now pacific time,utility,bf671c76-f377-4368-923e-3dcd358f76d5
15802,help me find a well reviewed hotel in dallas,travel,218aa2fc-35ed-42de-bbac-e8fcf270a9c1


In [5]:
# download the normal model
model = "en_core_web_md"
cli.download(model)
nlp = spacy.load(model)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [19]:
# make sure we have the old spacy model
nlp_old = spacy.load(model)
nlp_old.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [20]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [21]:
# create the 3 representations

# cv
cv = CountVectorizer()
cv.fit(train.message)
dtm = cv.transform(train.message)
print(dtm.shape)

# spacy vectors
docs = list(nlp_old.pipe(train.message, disable=['tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']))
dvs = [doc.vector for doc in docs]
dvs = np.array(dvs)
print(dvs.shape)

# the use embeddings
uses = list(nlp.pipe(train.message, disable=['tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']))
uvs = [doc.vector for doc in uses]
uvs = np.array(uvs)
print(uvs.shape)


(17500, 6330)
(17500, 300)
(17500, 300)


In [23]:
# fit the models

# count vects
knn = KNeighborsClassifier(3)
knn.fit(dtm.toarray(), train.domain)
preds_cv = knn.predict(dtm.toarray())

# spacy pretrained
knn = KNeighborsClassifier(3)
knn.fit(dvs, train.domain)
preds_spacy = knn.predict(dvs)

# universal sentence encodings
knn = KNeighborsClassifier(3)
knn.fit(uvs, train.domain)
preds_use = knn.predict(uvs)

In [ ]:
# put everything back
train[['cv', 'spacy', 'use']] = np.stack([preds_cv, preds_spacy, preds_use], axis=1)

In [ ]:
train.head(3)

In [ ]:
# accuracy
accuracy_score(train.category, train.cv)
accuracy_score(train.category, train.spacy)
accuracy_score(train.category, train.use)

In [ ]:
# create the submission file
submit = eval[['id', 'preds']]
submit.columns = ['id', 'category']

In [ ]:
# write the file
submit.to_csv("avs-submit.csv", index=False)